In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  


import atac_utils

In [ ]:
scores_abc = pd.read_csv("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/scores_l23abc.csv", 
                         index_col=0,
                        )
scores_abc['scores_c-a'] = scores_abc['scores_c'] - scores_abc['scores_a']
scores_abc

In [ ]:
%%time
adata = anndata.read("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw_multiome_l23.h5ad")
adata

In [ ]:
adata.X = adata.raw.X

In [ ]:
adata.obs['scores_a'] = scores_abc.loc[adata.obs.index,'scores_a'].copy()
adata.obs['scores_b'] = scores_abc.loc[adata.obs.index,'scores_b'].copy()
adata.obs['scores_c'] = scores_abc.loc[adata.obs.index,'scores_c'].copy()
adata.obs['scores_c-a'] = scores_abc.loc[adata.obs.index,'scores_c-a'].copy()

# fix cond incompatibility # P6NR vs P6
adata.obs['cond'] = adata.obs['cond'].apply(lambda x: x.replace('NR', ''))

In [ ]:
sample_labels = adata.obs['Sample'].values
time_labels = [s[:-1].replace('DR', '') for s in sample_labels]

adata.obs['sample'] = sample_labels #
adata.obs['time']   = time_labels

uniq_samples = natsorted(np.unique(sample_labels))
nr_samples = [s for s in uniq_samples if "DR" not in s]
dr_samples = [s for s in uniq_samples if "DR" in s]

uniq_conds = np.array(natsorted(np.unique(adata.obs['cond'].values)))
print(uniq_conds)

In [ ]:
condcode2cond = atac_utils.CONDCODE_TO_COND
condcode2cond

In [ ]:
sample_conditions = list(condcode2cond.values())
n_cond = len(sample_conditions)
sample_conditions

In [ ]:
%%time
adatas_pk = []
for cond_code, cond_name in condcode2cond.items():
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac/pmat_l23concensus_{cond_name}.h5ad'
    adata_pk = sc.read(f)
    adatas_pk.append(adata_pk)
    print(adata_pk)
    # break

In [ ]:
%%time

num_archetypal_cells = 300

offset = 1
SHUFFLE = False

l2fc_th = np.log2(2)
l2fc_th_s = np.log2(1.2)
alpha_th = 0.05

n_cond = len(sample_conditions)
n_peak = adatas_pk[0].shape[1]
qs_tensor   = np.zeros((n_cond,3,n_peak))  # 3 represents 3 pairwise comparisons (ca, ba, bc)
l2fc_tensor = np.zeros((n_cond,3,n_peak))

print(qs_tensor.shape, l2fc_tensor.shape)

for cond_code, cond_name in condcode2cond.items():
    # get sub
    adatasub = adata[adata.obs['cond']==cond_name]
    cells_rna = adatasub.obs.index.values
    
    # get A vs C 
    ranks_ac = adatasub.obs['scores_c-a'].rank()
    ranks_b  = adatasub.obs['scores_b'].rank()
    
    precond_a = ranks_ac <= num_archetypal_cells
    precond_c = ranks_ac > adatasub.shape[0] - num_archetypal_cells
    precond_b = ranks_b  > adatasub.shape[0] - num_archetypal_cells
    
    cond_a = np.all([ precond_a, ~precond_b, ~precond_c], axis=0)
    cond_b = np.all([~precond_a,  precond_b, ~precond_c], axis=0)
    cond_c = np.all([~precond_a, ~precond_b,  precond_c], axis=0)
    
    # get ATAC
    adata_pk = adatas_pk[cond_code]
    cells_atac = adata_pk.obs.index.values
        
    # SHUFFLE
    if SHUFFLE:
        cells_rna = cells_rna[np.random.choice(len(cells_rna), size=len(cells_rna), replace=False)]
        
    # get A, B, C 
    cells_a = np.intersect1d(cells_rna[cond_a], cells_atac)
    cells_b = np.intersect1d(cells_rna[cond_b], cells_atac)
    cells_c = np.intersect1d(cells_rna[cond_c], cells_atac)
        
    # peak size (500bp)
    mat_a = np.array(adata_pk[cells_a].X.todense()) 
    mat_b = np.array(adata_pk[cells_b].X.todense()) 
    mat_c = np.array(adata_pk[cells_c].X.todense()) 
    
    # size norm (CPM) between cells
    mat_a = mat_a/np.sum(mat_a, axis=1).reshape(-1,1)*1e6
    mat_b = mat_b/np.sum(mat_b, axis=1).reshape(-1,1)*1e6
    mat_c = mat_c/np.sum(mat_c, axis=1).reshape(-1,1)*1e6
    
    logmat_a = np.log2(mat_a+offset)
    logmat_b = np.log2(mat_b+offset)
    logmat_c = np.log2(mat_c+offset)
    
    # DEGs
    l2fc_ca = np.log2((np.mean(mat_c, axis=0)+offset)) - np.log2(np.mean(mat_a, axis=0)+offset)
    l2fc_ba = np.log2((np.mean(mat_b, axis=0)+offset)) - np.log2(np.mean(mat_a, axis=0)+offset)
    l2fc_bc = np.log2((np.mean(mat_b, axis=0)+offset)) - np.log2(np.mean(mat_c, axis=0)+offset)
    
    ts, ps_ca = stats.ttest_ind(logmat_c, logmat_a)
    ts, ps_ba = stats.ttest_ind(logmat_b, logmat_a)
    ts, ps_bc = stats.ttest_ind(logmat_b, logmat_c)
    
    rs, qs_ca, _, _ = multipletests(np.nan_to_num(ps_ca, nan=1).reshape(-1,), method='fdr_bh')
    rs, qs_ba, _, _ = multipletests(np.nan_to_num(ps_ba, nan=1).reshape(-1,), method='fdr_bh')
    rs, qs_bc, _, _ = multipletests(np.nan_to_num(ps_bc, nan=1).reshape(-1,), method='fdr_bh')
    
    # sig_cond
    qs_a   = np.minimum(qs_ca, qs_ba) # the better of the two
    qs_c   = np.minimum(qs_ca, qs_bc) # the better of the two
    qs_b   = np.minimum(qs_ba, qs_bc) # the better of the two

    l2fc_a = np.max([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
    l2fc_c = np.max([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
    l2fc_b = np.max([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change
    
    l2fc_as = np.min([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
    l2fc_cs = np.min([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
    l2fc_bs = np.min([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change

    cond_sig_a = np.all([-l2fc_ca > 0, -l2fc_ba > 0, l2fc_a > l2fc_th, qs_a < alpha_th], axis=0)
    cond_sig_c = np.all([ l2fc_ca > 0, -l2fc_bc > 0, l2fc_c > l2fc_th, qs_c < alpha_th], axis=0)
    cond_sig_b = np.all([ l2fc_ba > 0,  l2fc_bc > 0, l2fc_b > l2fc_th, qs_b < alpha_th, l2fc_bs > l2fc_th_s], axis=0)
    
    print(cond_code, cond_name, len(mat_a), len(mat_b), len(mat_c), 
          cond_sig_a.sum(), cond_sig_c.sum(), cond_sig_b.sum())
    
    # save this
    l2fc_tensor[cond_code, 0] = l2fc_ca
    l2fc_tensor[cond_code, 1] = l2fc_ba
    l2fc_tensor[cond_code, 2] = l2fc_bc
    
    qs_tensor[cond_code, 0] = qs_ca
    qs_tensor[cond_code, 1] = qs_ba
    qs_tensor[cond_code, 2] = qs_bc
    

In [ ]:
# %%time
# fout1 = '/u/home/f/f7xiesnm/v1_multiome/DAR_l23abc_qs_250411.npy'
# fout2 = '/u/home/f/f7xiesnm/v1_multiome/DAR_l23abc_l2fc_250411.npy'
# fout3 = '/u/home/f/f7xiesnm/v1_multiome/DAR_l23abc_cpm_tensor_250411.npy'

# np.save(fout1, qs_tensor)
# np.save(fout2, l2fc_tensor)

# check results and stats

In [ ]:
l2fc_th = np.log2(2)
l2fc_th_s = np.log2(1.2)
alpha_th = 0.05

In [ ]:
qs_ca   = qs_tensor[:,0,:]
qs_ba   = qs_tensor[:,1,:]
qs_bc   = qs_tensor[:,2,:]

l2fc_ca = l2fc_tensor[:,0,:]
l2fc_ba = l2fc_tensor[:,1,:]
l2fc_bc = l2fc_tensor[:,2,:]

In [ ]:
qs_a   = np.minimum(qs_ca, qs_ba) # the better of the two
qs_c   = np.minimum(qs_ca, qs_bc) # the better of the two
qs_b   = np.minimum(qs_ba, qs_bc) # the better of the two

l2fc_a = np.max([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
l2fc_c = np.max([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
l2fc_b = np.max([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change


l2fc_as = np.min([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
l2fc_cs = np.min([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
l2fc_bs = np.min([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change


cond_sig_a = np.all([-l2fc_ca > 0, -l2fc_ba > 0, l2fc_a > l2fc_th, qs_a < alpha_th], axis=0)
cond_sig_c = np.all([ l2fc_ca > 0, -l2fc_bc > 0, l2fc_c > l2fc_th, qs_c < alpha_th], axis=0)
# cond_sig_a = np.all([-l2fc_ca > 0, -l2fc_ba > 0, l2fc_a > l2fc_th, qs_a < alpha_th, l2fc_as > l2fc_th_s], axis=0)
# cond_sig_c = np.all([ l2fc_ca > 0, -l2fc_bc > 0, l2fc_c > l2fc_th, qs_c < alpha_th, l2fc_cs > l2fc_th_s], axis=0)
cond_sig_b = np.all([ l2fc_ba > 0,  l2fc_bc > 0, l2fc_b > l2fc_th, qs_b < alpha_th, l2fc_bs > l2fc_th_s], axis=0)


instances, counts_a = np.unique(cond_sig_a.sum(axis=0), return_counts=True)
instances, counts_c = np.unique(cond_sig_c.sum(axis=0), return_counts=True)
instances, counts_b = np.unique(cond_sig_b.sum(axis=0), return_counts=True)

print('num A genes for each cond:\t', cond_sig_a.sum(axis=1), np.any(cond_sig_a, axis=0).sum())
print('num C genes for each cond:\t', cond_sig_c.sum(axis=1), np.any(cond_sig_c, axis=0).sum())
print('num B genes for each cond:\t', cond_sig_b.sum(axis=1), np.any(cond_sig_b, axis=0).sum())

print('num A genes in num conds:\t',  counts_a[1:])
print('num C genes in num conds:\t',  counts_c[1:])
print('num B genes in num conds:\t',  counts_b[1:])

In [ ]:
a_any = np.sort(adata_pk.var[np.any(cond_sig_a, axis=0)].index.values)
a_all = np.sort(adata_pk.var[np.all(cond_sig_a, axis=0)].index.values)

c_any = np.sort(adata_pk.var[np.any(cond_sig_c, axis=0)].index.values)
c_all = np.sort(adata_pk.var[np.all(cond_sig_c, axis=0)].index.values)
ac_overlap = np.sort(adata_pk.var[np.logical_and(np.any(cond_sig_a, axis=0), np.any(cond_sig_c, axis=0))].index.values)

b_any = np.sort(adata_pk.var[np.any(cond_sig_b, axis=0)].index.values)
b_all = np.sort(adata_pk.var[np.all(cond_sig_b, axis=0)].index.values)

print('a any', a_any.shape)
print('a all', a_all.shape)

print('c any', c_any.shape)
print('c all', c_all.shape)
print('ac overlap', ac_overlap.shape)

print('b any', b_any.shape)
print('b all', b_all.shape)

In [ ]:
sample_conditions = np.array(list(condcode2cond.values()))
n_cond = len(sample_conditions)

nr_condcodes  = np.array([0,1,2,3,5,7,9])
dr_condcodes  = np.array([4,6,8,10])

nr_conditions = sample_conditions[nr_condcodes]
dr_conditions = sample_conditions[dr_condcodes]

nr_times = [6,8,10,12,14,17,21]
dr_times =        [12,14,17,21]
times = nr_times

print(sample_conditions)
print(nr_condcodes, nr_conditions)
print(dr_condcodes, dr_conditions)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(3*4,1*4), sharex=True, sharey=True)

num_uniq_a = np.sum(np.any(cond_sig_a,  axis=0))
num_uniq_b = np.sum(np.any(cond_sig_b, axis=0))
num_uniq_c = np.sum(np.any(cond_sig_c,  axis=0))

ax = axs[0]
ax.plot(times, cond_sig_a.sum(axis=1)[nr_condcodes], 
        '-o', color='C0', fillstyle='none', label=f'A DARs (unique n = {num_uniq_a:,})') 
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
# ax.set_ylim(ymin=-500, ymax=12500)
ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)
ax.set_ylabel('number of DARs')

ax = axs[1]
ax.plot(times, cond_sig_c.sum(axis=1)[nr_condcodes], 
        '-o', color='C2', fillstyle='none', label=f'C DARs (unique n = {num_uniq_c:,})') 
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
# ax.set_ylim(ymin=-500, ymax=12500)
ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)

ax = axs[2]
ax.plot(times, cond_sig_b.sum(axis=1)[nr_condcodes],
        '-o', color='C1', fillstyle='none', label=f'B DARs (unique n = {num_uniq_b:,})')
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)
# ax.set_ylim(ymin=0)
# ax.set_ylim(ymin=-50, ymax=2000)
ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)

fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(1*4,1*4)) # , sharex=True)

num_uniq_a = np.sum(np.any(cond_sig_a,  axis=0))
num_uniq_b = np.sum(np.any(cond_sig_b, axis=0))
num_uniq_c = np.sum(np.any(cond_sig_c,  axis=0))

ax.plot(times, cond_sig_a.sum(axis=1)[nr_condcodes], 
        '-o', color='C0', fillstyle='none', label=f'A DARs (unique n = {num_uniq_a:,})') 
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)

ax.plot(times, cond_sig_c.sum(axis=1)[nr_condcodes], 
        '-o', color='C2', fillstyle='none', label=f'C DARs (unique n = {num_uniq_c:,})') 
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)

ax.plot(times, cond_sig_b.sum(axis=1)[nr_condcodes],
        '-o', color='C1', fillstyle='none', label=f'B DARs (unique n = {num_uniq_b:,})')
ax.plot(times, [0]*len(times),'--o', color='gray', fillstyle='none', label=f'random DARs (n=0)', zorder=0)

ax.set_ylabel('number of DARs')
# ax.set_ylim(ymin=-500, ymax=12500)
ax.set_xticks(times)
ax.set_xticklabels(times)
ax.grid(False)
sns.despine(ax=ax)
ax.legend(bbox_to_anchor=(1,1))


fig.tight_layout()
# output = os.path.join(outfigdir, 'abc_dars_over_time.pdf') 
# powerplots.savefig_autodate(fig, output)

plt.show()

# check effect size 

In [ ]:
cond_sig_a_any = np.any(cond_sig_a, axis=0)
cond_sig_b_any = np.any(cond_sig_b, axis=0)
cond_sig_c_any = np.any(cond_sig_c, axis=0)

In [ ]:
%%time

# num_archetypal_cells = 300

# offset = 10
# SHUFFLE = False

# l2fc_th = np.log2(2)
# l2fc_th_s = np.log2(1.2)
# alpha_th = 0.05

# n_cond = len(sample_conditions)
# n_peak = adatas_pk[0].shape[1]
# qs_tensor   = np.zeros((n_cond,3,n_peak))  # 3 represents 3 pairwise comparisons (ca, ba, bc)
# l2fc_tensor = np.zeros((n_cond,3,n_peak))

# print(qs_tensor.shape, l2fc_tensor.shape)


l2fc_res_narrow = np.zeros((11,3,3))
l2fc_res_broad = np.zeros((11,3,3))
mmat_res_broad = np.zeros((11,3,3))
for i, (cond_code, cond_name) in enumerate(condcode2cond.items()):
    
    # get sub
    adatasub = adata[adata.obs['cond']==cond_name]
    cells_rna = adatasub.obs.index.values
    
    # get A vs C 
    ranks_ac = adatasub.obs['scores_c-a'].rank()
    ranks_b  = adatasub.obs['scores_b'].rank()
    
    precond_a = ranks_ac <= num_archetypal_cells
    precond_c = ranks_ac > adatasub.shape[0] - num_archetypal_cells
    precond_b = ranks_b  > adatasub.shape[0] - num_archetypal_cells
    
    cond_a = np.all([ precond_a, ~precond_b, ~precond_c], axis=0)
    cond_b = np.all([~precond_a,  precond_b, ~precond_c], axis=0)
    cond_c = np.all([~precond_a, ~precond_b,  precond_c], axis=0)
    
    # get ATAC
    adata_pk = adatas_pk[cond_code]
    cells_atac = adata_pk.obs.index.values
        
    # SHUFFLE
    if SHUFFLE:
        cells_rna = cells_rna[np.random.choice(len(cells_rna), size=len(cells_rna), replace=False)]
        
    # get A, B, C 
    cells_a = np.intersect1d(cells_rna[cond_a], cells_atac)
    cells_b = np.intersect1d(cells_rna[cond_b], cells_atac)
    cells_c = np.intersect1d(cells_rna[cond_c], cells_atac)
        
    # peak size (500bp)
    mat = np.array(adata_pk.X.todense()) 
    mat_a = np.array(adata_pk[cells_a].X.todense()) 
    mat_b = np.array(adata_pk[cells_b].X.todense()) 
    mat_c = np.array(adata_pk[cells_c].X.todense()) 
    
    # size norm (CPM) between cells
    mat = mat/np.sum(mat, axis=1).reshape(-1,1)*1e6
    mat_a = mat_a/np.sum(mat_a, axis=1).reshape(-1,1)*1e6
    mat_b = mat_b/np.sum(mat_b, axis=1).reshape(-1,1)*1e6
    mat_c = mat_c/np.sum(mat_c, axis=1).reshape(-1,1)*1e6
    
    mmat_a = np.log2(np.mean(mat_a, axis=0)+offset)
    mmat_b = np.log2(np.mean(mat_b, axis=0)+offset)
    mmat_c = np.log2(np.mean(mat_c, axis=0)+offset)
    
    # DEGs
    l2fc_a = (np.log2((np.mean(mat_a, axis=0)+offset)) - np.log2(np.mean(mat, axis=0)+offset))
    l2fc_b = (np.log2((np.mean(mat_b, axis=0)+offset)) - np.log2(np.mean(mat, axis=0)+offset))
    l2fc_c = (np.log2((np.mean(mat_c, axis=0)+offset)) - np.log2(np.mean(mat, axis=0)+offset))
    
    # narrow
    aa = np.mean(l2fc_a[cond_sig_a[cond_code]])
    ab = np.mean(l2fc_a[cond_sig_b[cond_code]])
    ac = np.mean(l2fc_a[cond_sig_c[cond_code]])
    
    ba = np.mean(l2fc_b[cond_sig_a[cond_code]])
    bb = np.mean(l2fc_b[cond_sig_b[cond_code]])
    bc = np.mean(l2fc_b[cond_sig_c[cond_code]])
    
    ca = np.mean(l2fc_c[cond_sig_a[cond_code]])
    cb = np.mean(l2fc_c[cond_sig_b[cond_code]])
    cc = np.mean(l2fc_c[cond_sig_c[cond_code]])
    
    l2fc_res_narrow[i] = np.array([
        [aa, ab, ac],
        [ba, bb, bc],
        [ca, cb, cc],
    ])
    
    # broad 
    aa = np.mean(l2fc_a[cond_sig_a_any])
    ab = np.mean(l2fc_a[cond_sig_b_any])
    ac = np.mean(l2fc_a[cond_sig_c_any])
    
    ba = np.mean(l2fc_b[cond_sig_a_any])
    bb = np.mean(l2fc_b[cond_sig_b_any])
    bc = np.mean(l2fc_b[cond_sig_c_any])
    
    ca = np.mean(l2fc_c[cond_sig_a_any])
    cb = np.mean(l2fc_c[cond_sig_b_any])
    cc = np.mean(l2fc_c[cond_sig_c_any])
    
    l2fc_res_broad[i] = np.array([
        [aa, ab, ac],
        [ba, bb, bc],
        [ca, cb, cc],
    ])
    
    # broad 
    aa = np.mean(mmat_a[cond_sig_a_any])
    ab = np.mean(mmat_a[cond_sig_b_any])
    ac = np.mean(mmat_a[cond_sig_c_any])
    
    ba = np.mean(mmat_b[cond_sig_a_any])
    bb = np.mean(mmat_b[cond_sig_b_any])
    bc = np.mean(mmat_b[cond_sig_c_any])
    
    ca = np.mean(mmat_c[cond_sig_a_any])
    cb = np.mean(mmat_c[cond_sig_b_any])
    cc = np.mean(mmat_c[cond_sig_c_any])
    
    mmat_res_broad[i] = np.array([
        [aa, ab, ac],
        [ba, bb, bc],
        [ca, cb, cc],
    ])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3*4,1*4), sharex=True, sharey=True)
for i in range(3):
    ax = axs[i]
    ax.plot(nr_times, l2fc_res_narrow[nr_condcodes,0,i])
    ax.plot(nr_times, l2fc_res_narrow[nr_condcodes,1,i])
    ax.plot(nr_times, l2fc_res_narrow[nr_condcodes,2,i])

    ax.plot(dr_times, l2fc_res_narrow[dr_condcodes,0,i], color='C0', alpha=0.5)
    ax.plot(dr_times, l2fc_res_narrow[dr_condcodes,1,i], color='C1', alpha=0.5)
    ax.plot(dr_times, l2fc_res_narrow[dr_condcodes,2,i], color='C2', alpha=0.5)

    ax.grid(False)
    ax.set_xticks(times)
    sns.despine(ax=ax)

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3*4,1*4), sharex=True, sharey=True)
for i in range(3):
    ax = axs[i]
    ax.plot(nr_times, l2fc_res_broad[nr_condcodes,0,i])
    ax.plot(nr_times, l2fc_res_broad[nr_condcodes,1,i])
    ax.plot(nr_times, l2fc_res_broad[nr_condcodes,2,i])

    ax.plot(dr_times, l2fc_res_broad[dr_condcodes,0,i], color='C0', alpha=0.5)
    ax.plot(dr_times, l2fc_res_broad[dr_condcodes,1,i], color='C1', alpha=0.5)
    ax.plot(dr_times, l2fc_res_broad[dr_condcodes,2,i], color='C2', alpha=0.5)

    ax.grid(False)
    ax.set_xticks(times)
    sns.despine(ax=ax)

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3*4,1*4), sharex=True, sharey=True)
for i in range(3):
    ax = axs[i]
    ax.plot(nr_times, mmat_res_broad[nr_condcodes,0,i])
    ax.plot(nr_times, mmat_res_broad[nr_condcodes,1,i])
    ax.plot(nr_times, mmat_res_broad[nr_condcodes,2,i])

    ax.plot(dr_times, mmat_res_broad[dr_condcodes,0,i], color='C0', alpha=0.5)
    ax.plot(dr_times, mmat_res_broad[dr_condcodes,1,i], color='C1', alpha=0.5)
    ax.plot(dr_times, mmat_res_broad[dr_condcodes,2,i], color='C2', alpha=0.5)

    ax.grid(False)
    ax.set_xticks(times)
    sns.despine(ax=ax)

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3*4,1*4), sharex=True, sharey=True)
for i in range(3):
    ax = axs[i]
    ax.plot(nr_times, mmat_res_broad[nr_condcodes,0,i])
    ax.plot(nr_times, mmat_res_broad[nr_condcodes,1,i])
    ax.plot(nr_times, mmat_res_broad[nr_condcodes,2,i])

    ax.grid(False)
    ax.set_xticks(times)
    sns.despine(ax=ax)

plt.show()

# get the full matrix (CPM - 5 fold) 

In [ ]:
%%time
n_type = 5

atac_tensor = np.zeros((n_cond, n_type, n_peak))
for i, cond in enumerate(sample_conditions):
    # get sub
    adatasub = adata[adata.obs['cond']==cond]
    cells_rna = adatasub.obs.index.values
    # x = adatasub.obsm['pca_p17on'][...,0]
    x = adatasub.obs['scores_c-a'].values
    cells_rna_type = pd.qcut(x, n_type, labels=False)
    
    # get ATAC
    adata_pk = adatas_pk[i]
    cells_atac = adata_pk.obs.index.values
    
    # per type
    for j in range(n_type):
        cells_j = np.intersect1d(cells_rna[cells_rna_type==j], cells_atac)
        atac_tensor[i,j] = np.array(np.sum(adata_pk[cells_j].X, axis=0))
        
# CPM
atac_tensor_cov = np.sum(atac_tensor, axis=2)
atac_tensor = atac_tensor/np.expand_dims(atac_tensor_cov, axis=2)*1e6 # .reshape(n_cond, n_type, -1) # *1e6
print(atac_tensor.shape)
np.sum(atac_tensor, axis=2)

In [ ]:
np.save(fout3, atac_tensor)